In [1]:
import torch
from torch_geometric.utils.convert import from_networkx
from pygod.detector import ANOMALOUS, SCAN, Radar, DOMINANT, OCGNN, GUIDE
from pygod.metric import eval_average_precision, eval_roc_auc, eval_f1, eval_precision_at_k, eval_recall_at_k
from pygod.generator import gen_contextual_outlier, gen_structural_outlier
import pickle

In [2]:
nD_graph_train = pickle.load(open('C:\\Users\\asus\\Documents\\tugas-akhir-latest-update\\data-prep-related\\graph_modeling\\fix-ipnyb/graph_modeling/te/nD_graph_train.pickle', 'rb'))
node_features_train = torch.load('C:\\Users\\asus\\Documents\\tugas-akhir-latest-update\\data-prep-related\\graph_modeling\\fix-ipnyb/graph_modeling/te/node_features_train.pt')
labels_train = torch.load('C:\\Users\\asus\\Documents\\tugas-akhir-latest-update\\data-prep-related\\graph_modeling\\fix-ipnyb/graph_modeling/te/labels_train.pt')

In [3]:
node_features_train

tensor([[4307.],
        [4307.],
        [4307.],
        ...,
        [4307.],
        [4307.],
        [4307.]], device='cuda:0')

In [4]:
nD_graph_test = pickle.load(open('C:\\Users\\asus\\Documents\\tugas-akhir-latest-update\\data-prep-related\\graph_modeling\\fix-ipnyb/graph_modeling/te/nD_graph_test.pickle', 'rb'))
node_features_test = torch.load('C:\\Users\\asus\\Documents\\tugas-akhir-latest-update\\data-prep-related\\graph_modeling\\fix-ipnyb/graph_modeling/te/node_features_test.pt')
labels_test = torch.load('C:\\Users\\asus\\Documents\\tugas-akhir-latest-update\\data-prep-related\\graph_modeling\\fix-ipnyb/graph_modeling/te/labels_test.pt')

In [5]:
pyG_train = from_networkx(nD_graph_train)
pyG_train = pyG_train.cuda()
pyG_train.x = node_features_train

pyG_test = from_networkx(nD_graph_test)
pyG_test.x = node_features_test

In [6]:
import pandas as pd
import numpy as np
test = pd.read_parquet("C:\\Users\\asus\\Documents\\tugas-akhir-latest-update\\data-prep-related\\graph_modeling\\fix-ipnyb\\test.parquet")
label = test['is_malware'].to_numpy()
# Switch the values 0 and 1 in the label variable
# label = np.where(label == 0, 1, np.where(label == 1, 0, label))
value_counts = np.unique(label, return_counts=True)
# Convert label to a tensor torch
label_tensor = torch.tensor(label, dtype=torch.float)
#convert label to tensor torch
# label_tensor = torch.tensor(label, dtype=torch.float)
value_counts

(array([0, 1]), array([2227,  222], dtype=int64))

In [22]:
train = pd.read_parquet("C:\\Users\\asus\\Documents\\tugas-akhir-latest-update\\data-prep-related\\graph_modeling\\fix-ipnyb\\train.parquet")
label_train = train['is_malware'].iloc[0].to_numpy()
# Switch the values 0 and 1 in the label variable
# label = np.where(label == 0, 1, np.where(label == 1, 0, label))
value_counts = np.unique(label_train, return_counts=True)
# Convert label to a tensor torch
label_tensor_train = torch.tensor(label, dtype=torch.float)
#convert label to tensor torch
# label_tensor = torch.tensor(label, dtype=torch.float)
value_counts

AttributeError: 'numpy.int32' object has no attribute 'to_numpy'

### ANOMALOUS

#### DOMINANT

In [12]:
dominant_model = DOMINANT(gpu=0, weight=0.02, num_layers=128, hid_dim=64, contamination=0.1, lr=0.001, verbose=3, epoch=100)

In [13]:
dominant_compile = dominant_model.fit(pyG_train)

Epoch 0000: Loss 87.5195 |  | Time 0.66
Epoch 0001: Loss 87.5194 |  | Time 0.59
Epoch 0002: Loss 87.5191 |  | Time 0.64
Epoch 0003: Loss 87.5185 |  | Time 0.56
Epoch 0004: Loss 87.5208 |  | Time 0.55
Epoch 0005: Loss 87.5179 |  | Time 0.78
Epoch 0006: Loss 87.5177 |  | Time 1.01
Epoch 0007: Loss 87.5172 |  | Time 0.65
Epoch 0008: Loss 87.5165 |  | Time 0.65
Epoch 0009: Loss 87.5156 |  | Time 0.57
Epoch 0010: Loss 87.5144 |  | Time 0.49
Epoch 0011: Loss 87.5131 |  | Time 0.51
Epoch 0012: Loss 87.5113 |  | Time 0.51
Epoch 0013: Loss 87.5088 |  | Time 0.53
Epoch 0014: Loss 87.5053 |  | Time 0.53
Epoch 0015: Loss 87.5004 |  | Time 0.54
Epoch 0016: Loss 87.4933 |  | Time 0.50
Epoch 0017: Loss 87.4827 |  | Time 0.50
Epoch 0018: Loss 87.4665 |  | Time 0.56
Epoch 0019: Loss 87.4410 |  | Time 0.73
Epoch 0020: Loss 87.4002 |  | Time 0.68
Epoch 0021: Loss 87.3333 |  | Time 0.66
Epoch 0022: Loss 87.2205 |  | Time 0.49
Epoch 0023: Loss 87.0261 |  | Time 0.51
Epoch 0024: Loss 86.6822 |  | Time 0.49


In [14]:
dominant_ip_pred_res, dominant_ip_score_res, dominant_ip_prob_res, dominant_ip_conf_res = dominant_compile.predict(data=pyG_test, label=label_tensor, return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(dominant_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss 0.0164 | AUC 0.4785 | Recall 0.0631 | Precision 0.0631 | AP 0.0831 | F1 0.0559 | Time 0.60
tensor([1]) tensor([2449])


### OCGNN

In [16]:
ocgnn_model = OCGNN(hid_dim=64, num_layers=64, weight_decay=0.02, 
                    contamination=0.1, lr=0.001, epoch=100, gpu=-1, batch_size=0, num_neigh=-1, 
                    beta=0.5, warmup=2, eps=0.001, verbose=3)

In [17]:
ocgnn_compile = ocgnn_model.fit(pyG_train)

Epoch 0000: Loss 0.0000 |  | Time 1.05
Epoch 0001: Loss 0.0006 |  | Time 0.99
Epoch 0002: Loss 0.0003 |  | Time 0.94
Epoch 0003: Loss 0.0005 |  | Time 0.98
Epoch 0004: Loss 0.0003 |  | Time 0.96
Epoch 0005: Loss 0.0003 |  | Time 0.97
Epoch 0006: Loss 0.0003 |  | Time 1.02
Epoch 0007: Loss 0.0003 |  | Time 1.11
Epoch 0008: Loss 0.0003 |  | Time 1.03
Epoch 0009: Loss 0.0003 |  | Time 0.99
Epoch 0010: Loss 0.0003 |  | Time 2.62
Epoch 0011: Loss 0.0003 |  | Time 2.49
Epoch 0012: Loss 0.0003 |  | Time 2.68
Epoch 0013: Loss 0.0003 |  | Time 1.98
Epoch 0014: Loss 0.0003 |  | Time 1.80
Epoch 0015: Loss 0.0003 |  | Time 2.06
Epoch 0016: Loss 0.0003 |  | Time 1.28
Epoch 0017: Loss 0.0003 |  | Time 1.54
Epoch 0018: Loss 0.0003 |  | Time 1.37
Epoch 0019: Loss 0.0003 |  | Time 1.29
Epoch 0020: Loss 0.0003 |  | Time 1.64
Epoch 0021: Loss 0.0003 |  | Time 1.43
Epoch 0022: Loss 0.0003 |  | Time 1.16
Epoch 0023: Loss 0.0003 |  | Time 1.09
Epoch 0024: Loss 0.0003 |  | Time 1.09
Epoch 0025: Loss 0.0003 |

In [18]:
ocgnn_ip_pred_res, ocgnn_ip_score_res, ocgnn_ip_prob_res, ocgnn_ip_conf_res = ocgnn_compile.predict(data=pyG_test, label=label_tensor, return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(ocgnn_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss 0.0000 | AUC 0.5056 | Recall 0.2568 | Precision 0.2568 | AP 0.0919 | F1 0.0460 | Time 0.33
tensor([0]) tensor([2449])


#### GUIDE

In [23]:
guide_model = GUIDE(num_layers=4, hid_a=64, hid_s=64, weight_decay=0.02, alpha=1, 
                    contamination=0.1, lr=0.001, epoch=100, gpu=-1, batch_size=0, num_neigh=-1, 
                    verbose=3)

In [24]:
guide_compile = guide_model.fit(pyG_train)

Epoch 0000: Loss 4817.6006 |  | Time 0.15
Epoch 0001: Loss 4594.2217 |  | Time 0.14
Epoch 0002: Loss 4389.7432 |  | Time 0.15
Epoch 0003: Loss 4209.9419 |  | Time 0.13
Epoch 0004: Loss 4037.5686 |  | Time 0.15
Epoch 0005: Loss 3860.9883 |  | Time 0.14
Epoch 0006: Loss 3683.4875 |  | Time 0.14
Epoch 0007: Loss 3503.2329 |  | Time 0.15
Epoch 0008: Loss 3317.2200 |  | Time 0.15
Epoch 0009: Loss 3142.2161 |  | Time 0.14
Epoch 0010: Loss 2984.1731 |  | Time 0.15
Epoch 0011: Loss 2816.3494 |  | Time 0.13
Epoch 0012: Loss 2642.0928 |  | Time 0.14
Epoch 0013: Loss 2458.7939 |  | Time 0.12
Epoch 0014: Loss 2272.1636 |  | Time 0.13
Epoch 0015: Loss 2075.0476 |  | Time 0.11
Epoch 0016: Loss 1867.0857 |  | Time 0.14
Epoch 0017: Loss 1648.5952 |  | Time 0.14
Epoch 0018: Loss 1439.9185 |  | Time 0.13
Epoch 0019: Loss 1224.2949 |  | Time 0.12
Epoch 0020: Loss 995.1100 |  | Time 0.13
Epoch 0021: Loss 746.9553 |  | Time 0.12
Epoch 0022: Loss 483.6100 |  | Time 0.12
Epoch 0023: Loss 204.3717 |  | Time 0

In [25]:
guide_ip_pred_res, guide_ip_score_res, guide_ip_prob_res, guide_ip_conf_res = guide_compile.predict(data=pyG_test, label=label_tensor, return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)

Test: Loss 0.0093 | AUC 0.4772 | Recall 0.0541 | Precision 0.0541 | AP 0.0879 | F1 0.0562 | Time 0.02
